# Day 10: Pipe Maze
I loved to have the Notebook for this ^^

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val pipeMap = Path("input.txt").readLines()

## Part 1
Obviously we must find the start.

In [ ]:
fun findStart(): Pair<Int, Int> {
    pipeMap.forEachIndexed { y, s ->
        if (s.contains('S')) {
            return s.indexOf('S') to y
        }
    }
    throw IllegalArgumentException("Unable to find starting position")
}

val start = findStart()
start

Onwards to finding the whole pipe. Once we have the total length we can divide it by 2 and got the solution.

For part 2 I changed what was `getPipeLength` to `getPipe` and returned the pipe segments instead of the length.

In [ ]:
val elementFilter = mapOf<Char, (Pair<Int, Int>) -> List<Pair<Int, Int>>>(
    'S' to { (cx, cy) -> listOf(cx - 1 to cy, cx to cy - 1, cx + 1 to cy, cx to cy + 1) },
    '|' to { (cx, cy) -> listOf(cx to cy - 1,cx to cy + 1) },
    '-' to { (cx, cy) -> listOf(cx - 1 to cy, cx + 1 to cy) },
    'L' to { (cx, cy) -> listOf(cx to cy - 1, cx + 1 to cy) },
    'J' to { (cx, cy) -> listOf(cx - 1 to cy, cx to cy - 1) },
    '7' to { (cx, cy) -> listOf(cx - 1 to cy, cx to cy + 1) },
    'F' to { (cx, cy) -> listOf(cx + 1 to cy, cx to cy + 1) },
)

fun findNext(current: Pair<Int, Int>, previous: Pair<Int, Int>?): Pair<Int, Int> {
    val (cx, cy) = current
    return elementFilter[pipeMap[cy][cx]]!!(current)
        .filterNot { it == previous }
        .filterNot { (x, y) -> x < 0 || y < 0 || x >= pipeMap[0].length || y >= pipeMap.size }
        .first { (x, y) ->
            x < cx && "-LFS".contains(pipeMap[y][x]) ||
                    y < cy && "|7FS".contains(pipeMap[y][x]) ||
                    x > cx && "-J7S".contains(pipeMap[y][x]) ||
                    y > cy && "|LJS".contains(pipeMap[y][x])
        }
}

fun getPipe(start: Pair<Int, Int>): List<Pair<Int, Int>> {
    var position = start
    var previousPosition: Pair<Int, Int>? = null
    var pipe = mutableListOf<Pair<Int, Int>>()
    do {
        var tmp = position
        position = findNext(position, previousPosition)
        previousPosition = tmp
        pipe.add(position)
    } while (position != start)
    
    return pipe
}

val pipe = getPipe(start)

pipe.size / 2

## Part 2
We need to find out what is *inside* the pipe and what not. I'm checking for a left-most element ...

In [ ]:
var (lowXX, lowXY) = pipe.filterNot { it == start }.reduce { a, b -> if (a.first < b.first) a else b }
lowXX to lowXY

Then what's on the right side of that element must be inside. So we get the first inside.

In [ ]:
val inside = when (pipeMap[lowXY][lowXX]) {
    'F' -> lowXX + 1 to lowXY + 1
    '|' -> lowXX + 1 to lowXY
    'L' -> lowXX + 1 to lowXY - 1
    else -> throw IllegalArgumentException("Wrong low x pipe element ${pipeMap[lowXY][lowXX]}")
}
inside

Next I want to go all the way around the pipe again, starting with the left-most element we found. For that purpose I will replace the `'S'` with the element that it stands for, so the algorithm doesn't stumble.

**Careful:** This does not check whether it's accessing coordinates outside the map. (I simply added a line of dots in the puzzle input in case of a NPE.)

In [ ]:
val (startX, startY) = start
val startElement = when {
    "-LF".contains(pipeMap[startY][startX-1]) && "|7F".contains(pipeMap[startY-1][startX]) -> 'J'
    "-LF".contains(pipeMap[startY][startX-1]) && "-J7".contains(pipeMap[startY][startX+1]) -> '-'
    "-LF".contains(pipeMap[startY][startX-1]) && "|LJ".contains(pipeMap[startY+1][startX]) -> '7'
    "|7F".contains(pipeMap[startY-1][startX]) && "-J7".contains(pipeMap[startY][startX+1]) -> 'L'
    "|7F".contains(pipeMap[startY-1][startX]) && "|LJ".contains(pipeMap[startY+1][startX]) -> '|'
    "-J7".contains(pipeMap[startY][startX+1]) && "|LJ".contains(pipeMap[startY+1][startX]) -> 'F'
    else -> throw IllegalArgumentException("WHAAAAT")
}
startElement

The next finds inner tiles along the pipe.

In [ ]:
// gives the options for both sides of the pipe. The following function need to determine which one is correct.
val insideFilter = mapOf<Char, (Pair<Int, Int>) -> List<List<Pair<Int, Int>>>>(
    '|' to { (cx, cy) -> listOf(listOf(cx - 1 to cy), listOf(cx + 1 to cy)) },
    '-' to { (cx, cy) -> listOf(listOf(cx to cy - 1), listOf(cx to cy + 1)) },
    'L' to { (cx, cy) -> listOf(listOf(cx + 1 to cy - 1), listOf(cx - 1 to cy, cx - 1 to cy + 1, cx to cy + 1)) },
    '7' to { (cx, cy) -> listOf(listOf(cx - 1 to cy + 1), listOf(cx to cy - 1, cx + 1 to cy - 1, cx + 1 to cy)) },
    'J' to { (cx, cy) -> listOf(listOf(cx - 1 to cy - 1), listOf(cx + 1 to cy, cx + 1 to cy + 1, cx to cy + 1)) },
    'F' to { (cx, cy) -> listOf(listOf(cx + 1 to cy + 1), listOf(cx - 1 to cy, cx - 1 to cy - 1, cx to cy - 1)) },
)

fun findInnerTiles(start: Pair<Int, Int>, inside: Pair<Int, Int>): Set<Pair<Int, Int>> {
    val startIndex = pipe.indexOf(start)
    return pipe.drop(startIndex).plus(pipe.take(startIndex))
        .asSequence()
        .windowed(2)
        .fold(mutableListOf(inside)) { insides, (p1, p2) ->
            val (p1x, p1y) = p1
            val (p2x, p2y) = p2
            val lastInsides = insides.takeLast(3)
            val c = if (pipeMap[p2y][p2x] == 'S') startElement else pipeMap[p2y][p2x]
            val insideOptions = insideFilter[c]!!(p2)
            insides.addAll(when {
                p2y == p1y -> insideOptions.first { l -> l.any { (_,y) -> lastInsides.map { it.second }.contains(y) } }
                p2x == p1x -> insideOptions.first { l -> l.any { (x) -> lastInsides.map { it.first }.contains(x) } }
                else -> throw IllegalArgumentException("couldn't find inner element")
            })
            insides
        }
        .filterNot { pipe.contains(it) }
        .toSet()
}

val initialInnerTiles = findInnerTiles(lowXX to lowXY, inside)
initialInnerTiles.size

Of course, that's not all inner tiles. There some tiles more inside. The following piece of code finds those tiles as well by recursively checking all neighboring tiles of the inside tiles that have been found.

In [ ]:
val allInnerTiles = mutableSetOf<Pair<Int, Int>>()
fun checkAllTiles(start: Pair<Int, Int>) {
    if (allInnerTiles.contains(start)) {
        return
    }
    
    allInnerTiles.add(start)
    val (x, y) = start
    listOf(x-1 to y, x to y-1, x+1 to y, x to y+1)
        .filterNot { pipe.contains(it) }
        .forEach { checkAllTiles(it) }
}
for (tile in initialInnerTiles) {
    checkAllTiles(tile)
}
allInnerTiles.size